# Part I. ETL Pipeline for Pre-Processing the Files

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [15]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [16]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data/'


filepath_list = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    filepath_list.append(file_path_list)
    
filepath_list = filepath_list[0]   

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [17]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in filepath_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [18]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


Working with the file <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [19]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [20]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [21]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. With Apache Cassandra you model the database tables on the queries you want to run.

## Below, I create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [22]:
"""
My planned schemas for the three tables:


Table 1
-------
Table Name: session_items
sessionId,
itemInSession,
artist,
song_title,
song_length,
Primary Key (sessionId, itemInSession)


Table 2
-------
Table Name: user_sessions
userid,
sessionid,
itemInSession,
artist,
song,
firstName,
lastName,
Primary Key (userid, sessionid, itemInSession)


Table 3
-------
Table Name: song_listens
song,
userId,
firstName,
lastName,
Primary Key (song, userId)

""";

In [23]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

## For this table I have chosen sessionId and itemInSession as the primary key. These two fields together will ensure a unique primary key for each record.
## sessionId is the partition key in this case - meaning that all records with the same sessionId will be stored on the same partition.

query = "CREATE TABLE IF NOT EXISTS session_items (sessionId int, itemInSession int, artist text, song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_items (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))    
        
query = "select artist, song_title, song_length from session_items WHERE sessionId = 33 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)        

Row(artist='Kanye West', song_title='Stronger', song_length=311.9277038574219)


In [24]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## For this table I have chosen to use userId, sessionId, itemInSession as the primary key.
## userId is the partition key - meaning records with the same userId will be stored on the same partition.
## sessionId is included as a clustering column, because the target query uses sessionId in the WHERE clause.
## itemInSession is also included as a clustering column, not because it is used in the WHERE clause, but because records returned from the query
## must be sorted by itemInSession.


query = "CREATE TABLE IF NOT EXISTS user_sessions (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'    
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_sessions (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))  
        
query = "select artist, song, itemInSession,firstName,lastName from user_sessions WHERE userId=53 AND sessionId=215"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)        

Row(artist='Dido', song="I'm No Angel", iteminsession=0, firstname='Celeste', lastname='Williams')
Row(artist='Lifehouse', song='You And Me (Wedding Version)', iteminsession=1, firstname='Celeste', lastname='Williams')
Row(artist='Gary U.S. Bonds', song='Angelyne', iteminsession=2, firstname='Celeste', lastname='Williams')
Row(artist='Throw Me The Statue', song='Noises', iteminsession=3, firstname='Celeste', lastname='Williams')


In [25]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## for this table I use song and userId as the primary key.
## song is the partition key - meaning all records for a paritcular song will be stored on the same partition.
## I have included userId in the primary key to ensure that users with the same first/last name can be distinguished.

query = "CREATE TABLE IF NOT EXISTS song_listens (song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'    
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listens (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))  
        
query = "select firstName,lastName,song from song_listens where song = 'You Never Let Go'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)  
                    

Row(firstname='Avery', lastname='Watkins', song='You Never Let Go')


### Drop the tables before closing out the sessions

In [26]:
## Dropping the table before closing out the sessions
query = "drop table session_items"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_listens"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()